Import

In [1]:
import sys
import os
import empyrical
import numpy as np
import pandas as pd
import configparser
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import dask.dataframe as dd
from dask.multiprocessing import get
from IPython.display import display
config = configparser.ConfigParser()
config.read('../../config.ini')
data_path = config.get('path', 'data_path')
local_path = config.get('path', 'local_path')

sys.path.append(local_path)


from shanf import apt 
from shanf.datakit.DtLoader import DtLoader 
from shanf.apt.Factor import Factor,IndexComp_Factor
from scripts import stats_utils as su
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import tushare as ts 
ts.set_token(config.get('token', 'tushare'))
pro = ts.pro_api()

NoSectionError: No section: 'path'

In [267]:
os.path.join(data_path,'processed/index/close.pkl')

'/Users/apple/Documents/qsquant/processed/index/close.pkl'

In [268]:
import datetime
# 假设你的 DataFrame 名为 df，包含 symbol, listdate 列
# 将 listdate 列转换为日期类型
basic_info = pd.read_csv(os.path.join(data_path,'source/stock/basic_info/basic_info.csv'),encoding='gbk',parse_dates=['listdate','delistdate'])  
basic_info['symbol'] = basic_info['symbol'].astype(str).apply(lambda x: x.zfill(6))
basic_info = basic_info.drop_duplicates(subset=['symbol'],keep='first')

df_suspend = pd.read_csv(os.path.join(data_path,'source/stock/basic_info/suspend.csv'),encoding = 'gbk',parse_dates=['ntradebegtime','ntradeendtime','resumetime'])
df_suspend.columns = df_suspend.columns.str.lower()
df_suspend = df_suspend[df_suspend['symbol'].notna()]
df_suspend['symbol'] = df_suspend['symbol'].astype(str).apply(lambda x: x.zfill(6))

### 定义初始样本池

In [269]:
def get_tradedays(start= '20060101',end = datetime.datetime.today().strftime('%Y%m%d')):
    tradedays = pro.trade_cal(exchange='',start_date=start,end_date=end).query('is_open==1')
    tradedays = pd.to_datetime(tradedays['cal_date']).sort_values().tolist()
    return tradedays


def get_tradeassets():
    return stock_fl.close.columns.tolist()

def convert_symbol(df, col):
    df[col] = df[col].astype(str).apply(lambda x: x.zfill(6))
    return df

index_comp_df = pd.read_csv(os.path.join(local_path,'doc/index_comp.csv'),encoding='gbk')
index_comp_df.columns = index_comp_df.columns.str.lower()
index_comp_df = convert_symbol(index_comp_df, 'symbol')
index_comp_df = convert_symbol(index_comp_df, 'selectesymbol')

In [270]:
stock_fl = DtLoader(field_root=os.path.join(data_path, 'processed/stock'))
tradedays = get_tradedays(end='20230901')
tradeassets = get_tradeassets()
tradeassets = [ i for i in tradeassets if i in list(basic_info['symbol'].values) and i not in  ['301329', '301456', '301503', '833751']]
stock_fl.params.update({'tradeassets':tradeassets,'tradedays':tradedays})
stock_fl.reload()
stock_fl.set_default_days_assets()

Blacklist
 - 净资产为负的个股 （最近一个报告期）
 - 次新股 （上市不足一年）
 - 退市
 - ST股

不可交易股
 - 下个交易日涨/跌停
 - 停牌

In [271]:
def get_new_listed_stocks():
    
    new_listed = pd.DataFrame(index=tradedays, columns=tradeassets)

    def calc_days_since_listdate(symbol):
        try:
            stock_listdate = basic_info.loc[basic_info['symbol'] == symbol, 'listdate'].values[0]
            return (new_listed.index - stock_listdate).days
        except:
            print(symbol)
            return None

    new_listed = new_listed.apply(lambda column: calc_days_since_listdate(column.name))
    new_listed.loc[:,:] = np.where(new_listed<=252,True,False)
        
    return new_listed

In [272]:
def get_delisted_stocks():
    delisted = pd.DataFrame(index=tradedays, columns=tradeassets)

    def calc_days_since_listdate(symbol):
        try:
            stock_delistdate = basic_info.loc[basic_info['symbol'] == symbol, 'delistdate'].values[0]
            if str(stock_delistdate)[:4]=='1900': 
                return np.ones(len(tradedays))*-1
            else:
                return (delisted.index - stock_delistdate).days
        except:
            print(symbol)
            return None

    delisted = delisted.apply(lambda column: calc_days_since_listdate(column.name))
    delisted.loc[:,:] = np.where(delisted>0,True,False)
    return delisted

In [273]:
def get_st_stocks():
    st = pd.DataFrame(index=tradedays, columns = tradeassets)

    df_name_chgs = pro.namechange(limit=10000000)
    df_st = df_name_chgs[df_name_chgs['change_reason'].isin(['ST', '*ST', '终止上市'])].sort_values('start_date')
    df_st.start_date = pd.to_datetime(df_st.start_date)
    df_st.end_date = pd.to_datetime(df_st.end_date)
    
    for row in df_st.itertuples():
        if row.ts_code[:-3] in tradeassets:
            start = row.start_date
            if row.end_date is not None:
                end = row.end_date
                st.loc[start:end,row.ts_code[:-3]] = 1 
            else :
                st.loc[start:,row.ts_code[:-3]] = 1
    
    st = st.replace({np.nan:False}).astype(bool)
    
    return st.astype(bool)

In [274]:
def get_ud_limit_stocks():
    # 下个交易日 涨/跌停
    stock_fl.close = stock_fl.close.replace({0:np.nan}).ffill()
    up_down_limit = stock_fl.close[stock_fl.close.pct_change().shift(-1).abs()>=0.095]

    up_down_limit = up_down_limit.replace({np.nan:False}).astype(bool)
    up_down_limit = up_down_limit.loc[tradedays,tradeassets]
    return up_down_limit

In [275]:
def get_neg_asset_stocks():
    pb_tmp = stock_fl.pb.fillna(method='ffill',limit = 20*4)
    neg_asset = stock_fl.pb[pb_tmp<0]
    
    neg_asset = neg_asset.replace({np.nan:False}).astype(bool)
    neg_asset = neg_asset.loc[tradedays,tradeassets]
    return neg_asset

In [276]:
def get_suspend_stocks():
    suspend =  pd.DataFrame(index=tradedays, columns = tradeassets)
    df_suspend.ntradebegtime = pd.to_datetime(df_suspend.ntradebegtime)
    df_suspend.resumetime = pd.to_datetime(df_suspend.resumetime)

    for row in df_suspend.itertuples():
        if row.symbol not in tradeassets: continue
        
        start = row.ntradebegtime
        
        if row.resumetime is not None :
            
            end = row.resumetime
            
            if row.resumetime > datetime.datetime.strptime('1900-01-01','%Y-%m-%d'):
                suspend.loc[start:end,row.symbol] = 1
            else :
                suspend.loc[start:,row.symbol] = 1
        else :
            suspend.loc[start:,row.symbol] = 1                

    suspend = suspend.replace({np.nan:False}).astype(bool)
        
    return suspend

In [277]:
def get_no_price():
    no_price = (stock_fl.close.isna())
    return no_price

剔除停牌等原因无法交易的股票

In [278]:
st = get_st_stocks()
new_listed = get_new_listed_stocks()
# neg_asset = get_neg_asset_stocks()
delisted = get_delisted_stocks()
suspend = get_suspend_stocks()
ud_limit = get_ud_limit_stocks()
no_price = get_no_price()

untradebale = (new_listed | st | suspend | ud_limit | no_price | delisted)

In [279]:
stock_pool = (~untradebale).astype(bool).astype(int).replace(0, np.nan)

In [281]:
stock_pool.to_pickle('../../tests/stock_pool.pkl')

In [282]:
stock_pool= pd.read_pickle('../../tests/stock_pool.pkl')

股票池确定

In [284]:
Factor.set_fl(stock_fl)
Factor.set_basic_info(basic_info)
Factor.stock_pool = stock_pool

风格因子

In [286]:
bm_price = pd.read_pickle(os.path.join(data_path,'processed/index/close.pkl'))['000300'].dropna().ffill()
bm_ret = np.log(bm_price).diff()
bm_ret = bm_ret.loc[tradedays]

In [292]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value
        
        
class Momentum(Factor):
    
    def _rolling_decay(self,_x,_d,func = np.sum):
        _x = _x[~np.isnan(_x)]
        _d = _d[-len(_x):]
        return func(_x * _d)
    
    def calculate(cls):
        return cls.RSTR * 1
    
    @lazyproperty
    def RSTR(self):        
        # Log return
        
        close = Factor.fl.close.ffill()
        log_ret = np.log(close).diff()
        excess_ret = log_ret.sub(bm_ret,axis=0)
        
        # lagged by 21 days
        lag = 21
        excess_ret = excess_ret.shift(lag)

        window = 504
        halflife = 126
        decay_weights = (0.5 ** (np.arange(window) / halflife))[::-1]
        
        rstr = self._dask_rolling_decay(excess_ret, window, decay_weights, min_periods=252, func=np.sum)
        return rstr

In [293]:
Momentum().RSTR

AttributeError: 'Momentum' object has no attribute '_dask_rolling_decay'

In [199]:
class Size(Factor):
    @lazyproperty
    def lncap(self):
        lncap = Factor.fl.negotiablemv
        lncap = self.prevent_0(lncap)
        lncap = np.log(lncap)
        return lncap
    
class NonLinearSize(Factor):
    def normalize(self,_m):
        return  _m.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
    
    def _regress(self,y, x, intercept, weights):
        if intercept:
            X = sm.add_constant(x)
        else:
            X = x
        model = sm.WLS(y, X, weights=weights)
        results = model.fit()
        return results.params[0], results.params[1]
    
    def _midcap_regress(self,mktcap):
        x = mktcap.dropna().values
        y = x**3
        alpha, beta = self._regress(y, x, intercept=True, weights=np.sqrt(x))
        resid = mktcap**3 - alpha - beta * mktcap
        return resid

    def MIDCAP(self):
        lncap = Size().LNCAP()
        midcap = lncap.apply(lambda x: self._midcap_regress(x) ,axis=1)
        return midcap
    

In [200]:
lncap = Size().lncap

In [20]:
midcap = NonLinearSize().MIDCAP()

In [21]:
from pyfinance.utils import rolling_windows

class BETA_HALPHA_HSIGMA(Factor):
    
    def _regress(self,y, x, intercept, weights):
        
        if intercept:
            X = sm.add_constant(x)
        else:
            X = x
        model = sm.WLS(y, X, weights=weights)
        results = model.fit()
        return results.params[0], results.params[1]
    
    def beta_halpha_hsigma(self):
        
        close = Factor.fl.close.ffill()
        log_ret = np.log(close).diff()
        excess_ret = log_ret.sub(bm_ret,axis=0)

        window = 252
        halflife = 63
        decay_weights = 0.5 ** (np.arange(window) / halflife)

        alphas = pd.DataFrame(index=excess_ret.index,columns=excess_ret.columns)
        betas = pd.DataFrame(index=excess_ret.index,columns=excess_ret.columns)
        sigma = pd.DataFrame(index=excess_ret.index,columns=excess_ret.columns)

        _rolling_ret = rolling_windows(excess_ret.values,window)
        _rolling_bm_ret = rolling_windows(bm_ret.values,window)

        for i, (y,x) in enumerate(zip(_rolling_ret,_rolling_bm_ret)):
            
            if np.any(np.isnan(x)):
                alpha,beta,resid = np.nan,np.nan,np.nan
                
            else:
                # 获取回归系数，其中第一个系数为常数项，第二个系数即为beta因子
                x = sm.add_constant(x)
                model = sm.WLS(y, x, weights=decay_weights).fit()

                alpha = model.params[0]
                beta = model.params[1]
                resid = np.std(model.resid,axis=0)

            alphas.loc[excess_ret.index[i+window-1]] = alpha
            betas.loc[excess_ret.index[i+window-1]] = beta
            sigma.loc[excess_ret.index[i+window-1]] = resid
            
        return (betas, alphas, sigma)

In [78]:
x = BETA_HALPHA_HSIGMA().beta_halpha_hsigma()

In [ ]:
class Growth(Factor):
    @lazyproperty
    def EGRO(self):
        egro = Factor.fl.EGRO
        return egro

    @lazyproperty
    def EYOY(self):
        netprofit = Factor.fl.netprofit_ttm
        eyoy = netprofit.pct_change(252)
        return eyoy
    
    @lazyproperty
    def SGRO(self):
        sgro = Factor.fl.SGRO
        return sgro
    
    @lazyproperty
    def SYOY(self):
        sales = Factor.fl.bizinco_ttm
        syoy = sales.pct_change(252)
        return syoy
   
    @lazyproperty 
    def CGRO(self):
        cgro = Factor.fl.cgro
        return cgro
    
    @lazyproperty
    def CYOY(self):
        cash = Factor.fl.mananetr
        cyoy = cash.pct_change(252)
        return cyoy
    
    @lazyproperty
    def ROEGRO(self):
        roe = Factor.fl.roe_ttm
        roegro = roe.pct_change(252)
        return roegro
    
    @lazyproperty
    def ROEYOY(self):
        roe = Factor.fl.roe_ttm
        roeyoy = roe.pct_change(252)
        return roeyoy
    
    @lazyproperty
    def EPSGRO(self):
        eps = Factor.fl.basiceps_ttm
        epsgro = eps.pct_change(252)
        return epsgro
    
    

In [264]:
class Valuation(Factor):
    
    def calculate(self):
        pass
    
    def _sync_days(self,_fin,tradedays):
        idx = _fin.index.union(tradedays)
        _fin = _fin.reindex(idx).ffill().fillna(0)
        _fin = _fin.loc[tradedays]
        return _fin
    
    @lazyproperty
    def BTOP(self):
        # mrq
        paresharrigh = self._sync_days(Factor.fl.paresharrigh,tradedays)
        prest = self._sync_days(Factor.fl.prest,tradedays)
        perbond = self._sync_days(Factor.fl.perbond,tradedays)
        totmktcap = Factor.fl.totmktcap.ffill()
        pb = (totmktcap*10000 / (paresharrigh-prest-perbond)).replace({np.inf:np.nan})
        btop = 1/pb
        return btop

In [27]:
close = Factor.fl.close.ffill()
negotiablemv = Factor.fl.negotiablemv.ffill()
prices=  close*stock_pool
prices.index.set_names(['date'], inplace=True)

In [239]:
import analyzer
import imp
imp.reload(analyzer)

<module 'analyzer' from '/Users/apple/Documents/GitHub/shanfolio/shanf/backtest/analyzer.py'>

In [262]:
factor = Valuation().BTOP
factor = factor*stock_pool

In [263]:
factor

symbol,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,873122,873152,873167,873169,873223,873305,873339,873527,873576,873593
tradedate,,,,,,,,,,,,,,,,,,,,,
2006-01-04,2.436659,2.572821,3.930257,1.427485,NaN,6.728206,4.316644,NaN,2.936649,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,2.452179,2.605948,NaN,1.436033,NaN,6.851659,NaN,NaN,2.988169,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-06,2.487100,2.589384,4.446162,1.453129,NaN,6.892810,4.810275,NaN,3.039689,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-09,2.479339,2.556258,4.530582,1.470224,NaN,7.139717,4.978319,NaN,3.153034,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-10,2.436659,2.622511,4.389881,1.461676,NaN,7.077990,4.841783,NaN,3.132426,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,0.575790,0.666825,NaN,NaN,0.737472,NaN,NaN,3.260759,NaN,1.229190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-27,0.580940,0.694953,NaN,NaN,0.764849,NaN,NaN,3.299024,NaN,1.260155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-28,0.581970,0.689619,NaN,NaN,0.744316,NaN,NaN,3.244359,NaN,1.252077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
basic_info = pd.read_csv(os.path.join(data_path,'source/stock/basic_info/basic_info.csv'),encoding='gbk',parse_dates=['listdate','delistdate'])  
basic_info['symbol'] = basic_info['symbol'].astype(str).apply(lambda x: x.zfill(6))
basic_info = basic_info.drop_duplicates(subset=['symbol'],keep='first')

In [242]:
basic_info.dropna(subset=['swlevel1name'],inplace=True)

In [256]:
a = analyzer.Analyzer(prices,negotiablemv,{'btop':factor},stock_pool)
a.init_bt(start='20130101',end='20230630',basic_info=basic_info)
a.eval_ic_all()
a.eval_perf_all()

,L_ann_pre_ret,L_sharpe_ratio,LS_ann_pre_ret,LS_sharpe_ratio
btop,0.0464,0.6853,-0.0384,0.4629


In [195]:
market_index_returns = bm_price.resample('M').last().pct_change().shift(-1)

In [248]:
from shanf.datakit import DtTransform
import imp 
imp.reload(DtTransform)

<module 'shanf.datakit.DtTransform' from '/Users/apple/Documents/Github/shanfolio/shanf/datakit/DtTransform.py'>

In [249]:
trans = DtTransform.DtTransform()

In [250]:
properties= {'basic_info':basic_info,'negotiablemv':negotiablemv,'sector_dummy':sector_dummy}

trans.set_properties(properties)

In [251]:
transforms = [trans.winsorize, trans.normalize,trans.fillna_by_sec_median, trans.neuturalize_sector_and_mktcap]

trans.run(factor, transforms, **kwargs)

symbol,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,873122,873152,873167,873169,873223,873305,873339,873527,873576,873593
tradedate,,,,,,,,,,,,,,,,,,,,,
2006-01-04,-0.189815,-0.36313,-0.443157,0.242132,-0.057673,-1.624864,-1.39232,-0.242087,-1.207905,-0.065347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,-0.123576,-0.291475,-0.016053,0.330456,-0.071979,-1.60947,-0.160147,-0.220907,-1.223385,-0.081353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-06,-0.124931,-0.260159,-0.592354,0.271808,-0.072724,-1.620653,-1.381013,-0.235688,-1.226675,-0.08178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-09,-0.111117,-0.27996,-0.608432,0.274227,-0.067003,-1.636674,-1.444715,-0.201301,-1.240818,-0.074975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-10,-0.093191,-0.293371,-0.576207,0.296346,-0.055255,-1.624172,-1.429034,-0.196667,-1.251558,-0.063166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,0.060052,1.128054,-0.093322,-0.041257,1.086065,-0.173023,-0.104962,-0.252029,-0.210243,-0.398819,...,-0.059087,-0.099194,-0.155463,-0.10688,-0.106872,-0.106984,-0.145992,-0.124802,NaN,-0.106179
2023-06-27,0.054307,1.129903,-0.100597,-0.060524,1.090797,-0.193783,-0.119441,-0.257317,-0.20506,-0.405766,...,-0.051371,-0.10787,-0.15848,-0.121183,-0.121177,-0.12128,-0.133495,-0.126722,NaN,-0.120594
2023-06-28,0.054515,1.125692,-0.106733,-0.072207,1.088289,-0.209142,-0.114048,-0.238798,-0.18176,-0.380925,...,-0.06077,-0.106017,-0.152952,-0.11572,-0.115713,-0.115812,-0.108497,-0.115301,NaN,-0.115118


In [253]:
factor

symbol,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,873122,873152,873167,873169,873223,873305,873339,873527,873576,873593
tradedate,,,,,,,,,,,,,,,,,,,,,
2006-01-04,0.410398,0.388678,0.254436,0.700533,NaN,0.148628,0.231661,NaN,0.340524,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,0.407801,0.383738,NaN,0.696363,NaN,0.145950,NaN,NaN,0.334653,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-06,0.402075,0.386192,0.224913,0.688170,NaN,0.145079,0.207888,NaN,0.328981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-09,0.403333,0.391197,0.220722,0.680168,NaN,0.140062,0.200871,NaN,0.317155,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-10,0.410398,0.381314,0.227797,0.684146,NaN,0.141283,0.206535,NaN,0.319241,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,1.736745,1.499643,NaN,NaN,1.355984,NaN,NaN,0.306677,NaN,0.813544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-27,1.721348,1.438946,NaN,NaN,1.307448,NaN,NaN,0.303120,NaN,0.793553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-28,1.718302,1.450077,NaN,NaN,1.343515,NaN,NaN,0.308227,NaN,0.798673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
